In [ ]:
import pandas as pd
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_bounds
from shapely.geometry import Point, Polygon, MultiPolygon,box
from scipy.spatial import cKDTree
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,WhiteKernel,ConstantKernel


# Etapa 1 : Obtener datos .kml de la ciudad de La Plata.

In [ ]:
gdf = gpd.read_file("laplata_cascourbano.kml", driver="KML")
display(gdf)

In [ ]:
# Chequear cuántas capas tiene el archivo kml
for layer in fiona.listlayers("laplata_cascourbano.kml"):
    print(layer)

In [ ]:
# Convertir el archivo kml a geojson
gdf.to_file("salida.geojson", driver='GeoJSON')

In [ ]:
# Plotear el archivo geojson
gdf = gpd.read_file("salida.geojson")
gdf.plot()

# Etapa 2 : Obtener datos de densidad poblacional de la ciudad de La Plata.

In [ ]:
# Dataset de densidad poblacional Argentina 2020 (1km resolución, ~4M puntos)
pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")

In [ ]:
# Función para extraer datos de una ciudad específica del dataset completo
def extract_city_data(city_name: str, bounds: dict) -> pd.DataFrame:

    df = pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")
    
    city_data = df[
        (df['X'] >= bounds['x_min']) & 
        (df['X'] <= bounds['x_max']) & 
        (df['Y'] >= bounds['y_min']) & 
        (df['Y'] <= bounds['y_max'])
    ]
    
    # Guardar subset
    output_file = f"{city_name.lower().replace(' ', '_')}_population_2020.csv"
    city_data.to_csv(output_file, index=False)
    
    print("Filas extraídas:", f"{len(city_data):,}")
    
    return city_data

In [ ]:
# Coordenadas aproximadas para La Plata
la_plata_bounds = {
   'x_min': -58.2,
   'x_max': -57.7,
   'y_min': -35.2,
   'y_max': -34.7
}

# Extraer datos
la_plata_data = extract_city_data("La Plata", la_plata_bounds)


In [ ]:
# Leer el archivo CSV de densidad de población de La Plata
df = pd.read_csv('la_plata_population_2020.csv')
    
# Crear geometría de puntos usando X,Y como longitud,latitud
geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]
    

In [ ]:
# Crear GeoDataFrame
CRS_4326 = 4326
gdf_densidad= gpd.GeoDataFrame(df, geometry=geometry, crs=CRS_4326)
    
# Agregar columna de densidad con nombre descriptivo
gdf_densidad['densidad_poblacional_lp'] = gdf_densidad['Z']
gdf_densidad


In [ ]:
#gdf.to_crs() #con esto cambio el sistema de referencia de coordenadas

In [ ]:
def export_to_geojson(gdf_densidad: gpd.GeoDataFrame, output_path:str = "la_plata_densidad.geojson") -> str:
        """
    Exporta un GeoDataFrame a formato GeoJSON
    
    Args:
        gdf_densidad: GeoDataFrame con datos de densidad
        output_path: Ruta de salida (default: "la_plata_densidad.geojson")
    
    Returns:
        str: Ruta del archivo exportado
    """
        
        gdf_densidad.to_file(output_path, driver='GeoJSON')
        print(f"Archivo exportado: {output_path}")
        return output_path

In [ ]:
export_to_geojson(gdf_densidad)

In [ ]:
# Visualización de la densidad poblacional de La Plata
fig, axes = plt.subplots(figsize=(6, 6))

# Gráfico de dispersión geográfica
scatter = axes.scatter(
    gdf_densidad['X'], gdf_densidad['Y'], c=gdf_densidad['Z'],
    cmap='inferno', alpha=0.6, s=10
)
axes.set_xlabel('Longitud')
axes.set_ylabel('Latitud')
axes.set_title('Distribución Geográfica de Densidad Poblacional de La Plata')
plt.colorbar(scatter, ax=axes, label='Densidad')

plt.tight_layout()
plt.savefig('densidad_la_plata.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Recortar el GeoDataFrame de densidad poblacional con el GeoDataFrame de La Plata

In [ ]:
def clip_density_to_urban_area(gdf_density: gpd.GeoDataFrame, gdf_geographical: gpd.GeoDataFrame) -> gpd.GeoDataFrame: 
    """
    Recorta los puntos de densidad que caen dentro del casco urbano de un límite geográfico dado.
    
    Args:
        gdf_density: GeoDataFrame con datos de densidad
        gdf_geographical: GeoDataFrame con límites geográficos

    Returns:
        gpd.GeoDataFrame: GeoDataFrame recortado a la zona urbana
    """
    # Usar uniones espaciales para mantener solo puntos dentro del casco urbano
    points_in_casco = gpd.sjoin(gdf_density, gdf_geographical, how='inner', predicate='intersects')
    
    # Limpiar columnas duplicadas del join 
    points_in_casco = points_in_casco.drop(columns=[col for col in points_in_casco.columns if col.endswith('_right')]) 
    
    return points_in_casco

In [ ]:
casco_urbano = clip_density_to_urban_area(gdf_densidad, gdf_geographical= gdf)

In [ ]:
casco_urbano

In [ ]:
# Convertir a Web Mercator (sist. de proyección cartográfica EPSG 3857)
casco_utm = casco_urbano.to_crs(epsg=3857)

In [ ]:
# Plotear usando un mapa base más clara y con mayor contraste
fig, ax = plt.subplots(figsize=(8, 8))

casco_utm.plot(column='densidad_poblacional_lp', 
               cmap='viridis',  
               alpha=0.8,    
               edgecolor='black',
               linewidth=2,
               ax=ax,
               legend=True,
               legend_kwds={'shrink': 1.0})

# Mapa base
ctx.add_basemap(ax, crs=casco_utm.crs.to_string(), 
                source=ctx.providers.CartoDB.Positron,  # Fondo más claro
                alpha=0.7)  

ax.set_title('Densidad Poblacional - Casco Urbano La Plata', fontsize=16)
ax.axis('on')
plt.tight_layout()
plt.show()

In [ ]:
# Visualización de la densidad poblacional de La Plata usando kriging

# Extraer coordenadas de los centroides,y valores desde el GeoDataFrame
coords =np.array([[point.x, point.y] for point in casco_utm.geometry.centroid]) 
values_density = casco_urbano['densidad_poblacional_lp'].values

print(f"Puntos de datos: {len(coords)}")    
print(f"Rango de valores: {values_density.min():.2f} - {values_density.max():.2f}")

# Crear grid de puntos para interpolación
bounds_casco_lp = casco_utm.total_bounds
grid_x, grid_y = np.mgrid[bounds_casco_lp[0]:bounds_casco_lp[2]:150j, bounds_casco_lp[1]:bounds_casco_lp[3]:150j]
grid_coords = np.column_stack([grid_x.ravel(), grid_y.ravel()])

#Definir kernel para el modelo de kriging
kernel = (ConstantKernel(1.0) * RBF(length_scale=1000.0) + 
          WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e3)))

# Crear modelo de kriging
gpr = GaussianProcessRegressor(kernel=kernel,
                                n_restarts_optimizer=10,
                                alpha=1e-04,
                                normalize_y=True)

# Entrenar el modelo
gpr.fit(coords, values_density)

#Predecir en el grid
grid_2_kriging, pred_std = gpr.predict(grid_coords, return_std=True)
grid_z = grid_2_kriging.reshape(grid_x.shape)
grid_uncertainty = pred_std.reshape(grid_x.shape)

# Visualización de la superficie Kriging interpolada
fig, ax = plt.subplots(figsize=(8, 10))

# Plotear superficie Kriging interpolada
contour = ax.contourf(grid_x, grid_y, grid_z, levels=30, cmap='viridis', alpha=0.7)

# Plotear los polígonos originales con bordes
casco_utm.plot(column='densidad_poblacional_lp', 
               cmap='viridis',
               alpha=0.5,
               edgecolor='black',
               linewidth=1.0,
               ax=ax)

# Plotear los puntos centroides
scatter = ax.scatter(coords[:, 0], coords[:, 1],
                    c=values_density,
                    cmap='viridis',
                    s=30,
                    edgecolors='black',
                    linewidths=1,
                    zorder=5)

#Mapa base más sutil
ctx.add_basemap(ax, crs=casco_utm.crs.to_string(),
               source=ctx.providers.CartoDB.Positron,
               alpha = 0.9)

# Agregar colorbar para el scatter
plt.colorbar(scatter, ax=ax, label='Densidad poblacional',shrink=0.7, aspect=25)

# Agregar título y etiquetas
ax.set_title('Interpolación Kriging - Densidad Poblacional La Plata')
ax.set_xlabel('X (UTM)')
ax.set_ylabel('Y (UTM)')

plt.tight_layout()
plt.show()

In [ ]:
def export_to_geotiff(grid_data: np.ndarray, gdf: gpd.GeoDataFrame, filename: str, crs_epsg: int = 3857):
    """
    Exporta una grilla (raster 2D) a un archivo GeoTIFF usando los límites geográficos de un GeoDataFrame.
    
    Parámetros:
    - grid_data: np.ndarray 2D, la grilla interpolada (como grid_z)
    - gdf: GeoDataFrame con CRS definido
    - filename: str, nombre de salida
    - crs_epsg: entero del sistema de coordenadas (ej. 3857)
    """
    bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
    height, width = grid_data.shape

    # Calcula la transformación georreferenciada
    transform = from_bounds(bounds[0], bounds[1], bounds[2], bounds[3], width, height)

    with rasterio.open(
        filename,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=grid_data.dtype,
        crs=CRS.from_epsg(crs_epsg),
        transform=transform,
        compress='lzw'
    ) as dst:
        dst.write(grid_data, 1)
        dst.set_band_description(1, 'Densidad Poblacional Kriging')

    print(f"Archivo exportado correctamente: {filename}")


In [ ]:
# Exportar superficie interpolada
export_to_geotiff(grid_z, casco_utm, 'kriging_densidad_poblacional.tif', 3857)

# Etapa 3 : superposición de mapas de densidad y peligrosidad de inundación de la ciudad de La Plata (QGIS).

In [ ]:
# Verificaión de los sistemas de referencia espacial
gdf_mapa = gpd.read_file("salida.geojson")
print(gdf.crs)
print(gdf_mapa.crs)